In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
import networkx as nx 
import osmnx as ox 
import numpy as np 
import seaborn as sns 
import pandas as pd
import networkx as nx
from shapely.geometry import Point, LineString
import pickle
import matplotlib.pyplot as plt
import folium
import pandas as pd
import networkx as nx
from shapely.geometry import Point, LineString
import pickle
from tqdm import tqdm 

pd.set_option('display.max_columns', None)

In [ ]:
# Read in the road network graph

with open(r"C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\pickles\BaseGraph0_3NOR_wagrades.pickle", 'rb') as file:
    G = pickle.load(file)

In [3]:
# create a dictionary mapping node IDs to (x, y) coordinates
node_coords = {node: (data['x'], data['y']) for node, data in G.nodes(data=True)}

In [ ]:
#Plot Graph

ox.plot_graph(G, figsize = (30, 15), node_color="red", edge_color = "gray", bgcolor = "white", node_size=5, edge_linewidth = 1)

In [ ]:
# Create an empty dataframe to store node attributes
node_df = pd.DataFrame(columns=['node', "osmid", "latitude", "longitude", 'gradient', "isBridge", "isTunnel", "roadclass", "geometry"])

# Iterate over all nodes in the network
# Note: Generates a lot of output - clear cell output after the code has run - must be a better way to this 
for node in G.nodes:
    # Get the gradient attribute of the node and add it to the dataframe
    osmid = G.nodes[node]["osmid"]
    lat = G.nodes[node]["x"]
    lon = G.nodes[node]["y"]
    isBridge = G.nodes[node]["isBridge"]
    isTunnel = G.nodes[node]["isTunnel"]
    roadclass = G.nodes[node]["roadclass"]
    gradient = G.nodes[node]['elevation']
    geo = G.nodes[node]["geometry"]

    node_df = node_df.append({"osmid": osmid, 'node': node, "latitude": lon, "longitude": lat, 'gradient': gradient, "isBridge": isBridge, "isTunnel": isTunnel, "roadclass": roadclass, "geometry": geo}, ignore_index=True)

In [6]:
# Convert the edge data to a pandas DataFrame
edge_data = G.edges(data=True)
edge_df = pd.DataFrame(edge_data, columns=['source', 'target', 'data_dict'])

# Extract the data into separate columns
edge_df = pd.concat([edge_df.drop('data_dict', axis=1), edge_df['data_dict'].apply(pd.Series)], axis=1)

In [7]:
# Convert the dataframes to geodataframes, using the "geometry" column as the geometry
edge_gdf = gpd.GeoDataFrame(edge_df, geometry="geometry")
node_gdf = gpd.GeoDataFrame(node_df, geometry="geometry")

In [ ]:
import matplotlib.pyplot as plt

# Create a scatter plot of the node locations
fig, ax = plt.subplots(figsize=(30, 15))
node_gdf.plot(ax=ax, markersize=1, color = 'red')

# Add a title and axis labels
ax.set_title("Node Locations")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

print("To many nodes on the same locations...")
# Display the plot
plt.show()

In [ ]:
# Import Traffic Counts:
# Load volume dataset
csv_pd=pd.read_excel(r"C:\Users\anton\Desktop\Master-Oppgave\Anton\Kode\All_Roads\CSV\all_roads_volume.xlsx")
# csv_pd.drop(columns='Unnamed: 0', inplace=True)
csv_pd.head(3)

In [ ]:
#Removing all duaplicates, in this case we only need one observation per registration point to visualize

csv_pd_woDup = csv_pd.copy().drop_duplicates('id').reset_index(drop=True)
csv_pd_woDup.head(3)

In [ ]:
# Remove volume1 and volume2

csv_pd_woDup['geometry'] = csv_pd_woDup.apply(lambda x: Point(x['lon'], x['lat']), axis=1)
csv_pd_woDup.head(3)

In [ ]:
traffic_counts = gpd.GeoDataFrame(csv_pd_woDup, geometry="geometry")
points_df = traffic_counts.copy()
traffic_counts.head(3)

In [ ]:
# set style to default
plt.style.use('ggplot')

# set up the plot
fig, ax = plt.subplots(figsize=(30, 15))

node_gdf.plot(ax=ax, color='black', markersize=7)
edge_gdf.plot(ax=ax, color='red')
traffic_counts.plot(ax=ax, color='blue', markersize=30)
ax.legend(['Nodes', 'Edges', 'Traffic_Counts'])
ax.set_title('Network with Traffic Counts in Blue', fontweight='bold', fontsize=25)

# display the plot
plt.show()

In [ ]:
# Create a folium map object centered on the first node in the node geodataframe
map_obj = folium.Map(location=[node_gdf.geometry.iloc[0].y, node_gdf.geometry.iloc[0].x], zoom_start=10)

# Create a feature group for the edges
edge_fg = folium.FeatureGroup(name="Edges")

# Add each edge as a line to the feature group, with a popup displaying the edge weight
for i, row in edge_gdf.iterrows():
    popup_str = f"Source: {row['source']}<br>Target: {row['target']}<br>Edge ID: {row['geometry']}"
    folium.PolyLine(locations=[(row.geometry.coords[0][1], row.geometry.coords[0][0]),
                                (row.geometry.coords[1][1], row.geometry.coords[1][0])],
                    popup=popup_str,
                    tooltip=f"Edge ({row['source']}, {row['target']})").add_to(edge_fg)

# Add the edge feature group to the map
edge_fg.add_to(map_obj)

# Add a layer control to the map, allowing the user to toggle the visibility of the edge and node feature groups
folium.LayerControl().add_to(map_obj)

# Display the map
map_obj

In [ ]:
import folium
import networkx as nx

# Create a folium map centered on the first point in points_df
m = folium.Map(location=[points_df.iloc[0]['lat'], points_df.iloc[0]['lon']], zoom_start=10)


# Add the points to the map
for _, row in points_df.iterrows():
    folium.Marker(location=[row['lat'], row['lon']], popup=(row['name'], row['id'])).add_to(m)

# Display the map
m


In [ ]:
to_exclude=[
    '', '','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','', 

In [ ]:
import plotly.graph_objs as go

# Create the map trace
map_trace = go.Scattermapbox(
    lat=points_df['lat'],
    lon=points_df['lon'],
    mode='markers',
    marker=dict(size=10, color='blue'),
    text=points_df['name'],
    hovertemplate="%{text}<br>Latitude: %{lat:.2f}<br>Longitude: %{lon:.2f}<extra></extra>"
)

# Create the graph trace
graph_trace = go.Bar(
    x=points_df['name'],
    y=points_df['volume'],
    marker=dict(color='green'),
    hovertemplate="%{x}<br>Volume: %{y}<extra></extra>"
)

# Create the subplot figure
fig = go.Figure(
    data=[map_trace, graph_trace],
    layout=go.Layout(
        title='Map and Volume Bar Chart',
        grid=dict(rows=1, columns=2),
        mapbox=dict(
            center=dict(lat=points_df['lat'].mean(), lon=points_df['lon'].mean()),
            zoom=10,
            style='open-street-map'
        ),
        xaxis=dict(title='Point Name'),
        yaxis=dict(title='Volume')
    )
)

# Display the subplot
fig.show()


In [ ]:
import mplleaflet

# set style to default
plt.style.use('ggplot')

# set up the plot
fig, ax = plt.subplots(figsize=(30, 15))

node_gdf.plot(ax=ax, color='black', markersize=7)
edge_gdf.plot(ax=ax, color='red')
traffic_counts.plot(ax=ax, color='blue', markersize=30)
ax.legend(['Nodes', 'Edges', 'Traffic_Counts'])
ax.set_title('Network with Traffic Counts in Blue', fontweight='bold', fontsize=25)

# convert the plot to an interactive map
mplleaflet.show(fig=fig)
